In [1]:
%matplotlib inline

import pandas as pd
from pathlib import Path
from biothings_client import get_client

from data_tools import graphs as gt
from data_tools.wiki import get_curi_xrefs
from data_tools import obo_processing as ot
from data_tools.df_processing import expand_col_on_char, expand_split_col, combine_group_cols_on_char

/home/mmayers/software/miniconda3/envs/ml/lib/python3.7/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
data_dir = Path('../2_pipeline/00_download_data/out/').resolve()
this_name = '06_HPO_Annotations'
out_dir = Path('../2_pipeline/').joinpath(this_name).joinpath('out').resolve()

In [3]:
if not out_dir.exists():
    out_dir.mkdir(parents=True)

In [4]:
prev_nodes = pd.read_csv('../2_pipeline/05c_Computing_GO_Regulation_Edges/out/nodes.csv', dtype=str)
prev_edges = pd.read_csv('../2_pipeline/05c_Computing_GO_Regulation_Edges/out/edges.csv', dtype=str)

## Get previous phenotypes and xrefs and see what we can match to HPO

In [5]:
pheno_nodes = prev_nodes.query('label == "Phenotype"')

In [6]:
nw_to_xref = expand_col_on_char(pheno_nodes, 'xrefs', '|', False)[['id', 'xrefs']]

In [7]:
nw_to_xref['nw_curi'] = nw_to_xref['id'].apply(lambda s: s.split(':')[0])
nw_to_xref['xref_curi'] = nw_to_xref['xrefs'].apply(lambda s: s.split(':')[0] if type(s) == str else s)

nw_to_xref[['nw_curi', 'xref_curi']].stack().unique()

array(['HP', 'MESH', 'UMLS', 'MONDO', 'OMIM', 'SNOMED', 'WD'],
      dtype=object)

In [8]:
hp_nodes = ot.get_ontology_nodes(data_dir.joinpath('hpo.obo'))
hp_struct = ot.get_ontology_edges(data_dir.joinpath('hpo.obo'))

In [9]:
hpo_to_xref = expand_col_on_char(hp_nodes[['id', 'xref']], 'xref', '|', False)
hpo_to_xref['xref_curi'] = hpo_to_xref['xref'].apply(lambda s: s.split(':')[0] if type(s) == str else s)

hpo_to_xref['xref_curi'].unique()

array(['UMLS', 'MSH', 'SNOMEDCT_US', 'MEDDRA', 'Fyler', 'NCIT', nan,
       'ICD-10', 'EPCC', 'ICD-O', 'DOID', 'MP', 'MPATH', 'PMID', 'pmid',
       'http', 'RGD', 'MeSH', 'DOI', 'ORPHA', 'HP', 'ICD-9'], dtype=object)

In [10]:
# Fix the curis so they're common between our network and HPO
hpo_to_xref['xref'] = (hpo_to_xref['xref'].str.replace('MSH:', 'MESH:')
                                          .str.replace('SNOMEDCT_US:', 'SNOMED:')
                                          .str.replace('MeSH:', 'MESH:'))
hpo_to_xref['xref_curi'] = hpo_to_xref['xref'].apply(lambda s: s.split(':')[0] if type(s) == str else s)


common_curi = set(nw_to_xref[['nw_curi', 'xref_curi']].stack().unique()) & set(hpo_to_xref['xref_curi'].unique())
common_curi

{'HP', 'MESH', 'SNOMED', 'UMLS'}

In [11]:
hpo_in_nw = nw_to_xref.query('nw_curi == "HP"')['id'].unique()

new_hpo = hpo_to_xref.query('id not in @hpo_in_nw and xref_curi in @common_curi')
len(new_hpo), new_hpo['id'].nunique()

(17321, 10858)

Look at what terms we can re-align with HPO IDs thanks to these xrefs

In [12]:
match_ids = nw_to_xref.query('nw_curi != "HP" and nw_curi in @common_curi or xref_curi in @common_curi')[['id', 'xrefs']].stack().unique()

found_pheno = new_hpo.query('xref in @match_ids').rename(columns={'id': 'hp_id'})
found_pheno['xref_curi'].value_counts()

MESH      1040
UMLS        11
SNOMED       5
Name: xref_curi, dtype: int64

About 10% were found to already be in the network, mostly via MESH cross-references.

In [13]:
pheno_reid_map = found_pheno.merge(nw_to_xref.query('nw_curi != "HP"').rename(columns={'id': 'nw_id'}),
                                   left_on='xref', right_on='xrefs')[['hp_id', 'nw_id']]

In [14]:
len(prev_nodes)

783901

In [15]:
changed_ids = pheno_reid_map['hp_id'].unique()

In [16]:
prev_edges = gt.re_id_edges(prev_edges, pheno_reid_map, 'nw_id', 'hp_id')
prev_nodes = gt.re_id_nodes(prev_nodes, pheno_reid_map, 'nw_id', 'hp_id', False)

In [17]:
prev_nodes.query('id in @changed_ids')

,id,name,label,xrefs
574185,HP:0000003,multicystic dysplastic kidney,Phenotype,MESH:D021782
574188,HP:0000011,neurogenic bladder,Phenotype,MESH:D001750
574189,HP:0000015,bladder diverticulum,Phenotype,MESH:C562406|OMIM:109820
574193,HP:0000023,inguinal hernia,Phenotype,MESH:D006552
574194,HP:0000024,prostatitis,Phenotype,MESH:D011472
...,...,...,...,...
575917,HP:0100890,choledochal cyst,Phenotype,MESH:D015529|OMIM:603003
575923,HP:0200008,Cronkhite-Canada syndrome,Phenotype,MESH:D044483|OMIM:175500
575924,HP:0200022,choroid plexus papilloma,Phenotype,MESH:D020288|OMIM:260500
575925,HP:0200023,priapism,Phenotype,MESH:D011317


In [18]:
# get_official hpo names into the network
hpid_to_name = hp_nodes.set_index('id')['name'].to_dict()

prev_nodes['name'] = prev_nodes['id'].map(hpid_to_name).fillna(prev_nodes['name'])

In [19]:
prev_nodes.query('id in @changed_ids')

,id,name,label,xrefs
574185,HP:0000003,Multicystic kidney dysplasia,Phenotype,MESH:D021782
574188,HP:0000011,Neurogenic bladder,Phenotype,MESH:D001750
574189,HP:0000015,Bladder diverticulum,Phenotype,MESH:C562406|OMIM:109820
574193,HP:0000023,Inguinal hernia,Phenotype,MESH:D006552
574194,HP:0000024,Prostatitis,Phenotype,MESH:D011472
...,...,...,...,...
575917,HP:0100890,Cyst of the ductus choledochus,Phenotype,MESH:D015529|OMIM:603003
575923,HP:0200008,Intestinal polyposis,Phenotype,MESH:D044483|OMIM:175500
575924,HP:0200022,Choroid plexus papilloma,Phenotype,MESH:D020288|OMIM:260500
575925,HP:0200023,Priapism,Phenotype,MESH:D011317


## Bring in the Annotations

In [20]:
tab_cols = ['db', 'db_object_id', 'db_name', 'qualifier', 'hp_id', 'db_reference', 'evidence_code', 'onset_modifier', 
            'frequency', 'sex', 'modifier', 'aspect', 'date_created', 'assigned_by']

hp_anno = pd.read_csv(data_dir.joinpath('phenotype_annotation.tab'), header=None, 
                       sep='\t', dtype=str, names=tab_cols)
hp_gene = pd.read_csv(data_dir.joinpath('ALL_SOURCES_ALL_FREQUENCIES_genes_to_phenotype.txt'), 
                       sep='\t', header=None, comment='#', dtype=str,
                       names=['gene_id', 'gene_symbol', 'hp_name', 'hp_id'])

In [21]:
hp_name_map = hp_nodes.set_index('id')['name'].to_dict()
hp_anno['hp_name'] = hp_anno['hp_id'].map(hp_name_map)

In [22]:
len(hp_anno), len(hp_gene)

(166084, 142947)

In [23]:
hp_anno.head(2)

,db,db_object_id,db_name,qualifier,hp_id,db_reference,evidence_code,onset_modifier,frequency,sex,modifier,aspect,date_created,assigned_by,hp_name
0,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0000252,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN,Microcephaly
1,DECIPHER,1,Wolf-Hirschhorn Syndrome,NaN,HP:0001249,DECIPHER:1,IEA,NaN,NaN,NaN,P,WOLF-HIRSCHHORN SYNDROME,HPO:skoehler,NaN,Intellectual disability


In [24]:
hp_gene.head(2)

,gene_id,gene_symbol,hp_name,hp_id
0,8192,CLPP,Seizures,HP:0001250
1,8192,CLPP,Short stature,HP:0004322


## Examine Genes

Genes are using entrez gene_id, so looking for overlap to the current network should be stratightforward

In [25]:
hp_gene['gene_id'] = 'NCBIGene:' + hp_gene['gene_id']
hp_genes = hp_gene['gene_id'].unique()

prev_nodes.query('id in @hp_genes')

,id,name,label,xrefs
338691,NCBIGene:100,ADA,Gene,ENSG:ENSG00000196839|HGNC:186|NCBIGene:100|OMI...
338693,NCBIGene:10000,AKT3,Gene,ENSG:ENSG00000117020|ENSG:ENSG00000275199|HGNC...
338712,NCBIGene:10002,NR2E3,Gene,ENSG:ENSG00000278570|HGNC:7974|NCBIGene:10002|...
338731,NCBIGene:100033413,SNORD116-1,Gene,HGNC:33067|NCBIGene:100033413|OMIM:605436|SYM:...
338854,NCBIGene:10008,KCNE3,Gene,ENSG:ENSG00000175538|HGNC:6243|NCBIGene:10008|...
...,...,...,...,...
461404,NCBIGene:999,CDH1,Gene,ENSG:ENSG00000039068|HGNC:1748|NCBIGene:999|OM...
461405,NCBIGene:9990,SLC12A6,Gene,ENSG:ENSG00000140199|HGNC:10914|NCBIGene:9990|...
461407,NCBIGene:9992,KCNE2,Gene,ENSG:ENSG00000159197|HGNC:6242|NCBIGene:9992|O...
461408,NCBIGene:9993,DGCR2,Gene,ENSG:ENSG00000070413|HGNC:2845|NCBIGene:9993|O...


In [26]:
hp_gene['gene_id'].nunique()

4016

In [27]:
node_ids = set(prev_nodes['id'])
set(hp_gene['gene_id']) - node_ids

{'NCBIGene:109580095'}

Only a very small number are missing... Don't think we'll add them, but let's at least look at what they

In [28]:
mg = get_client('gene')
mg.getgenes([g.split(':')[-1] for g in list(set(hp_gene['gene_id']) - node_ids)], fields=['name', 'symbol', 'uniprot'])

querying 1-1...done.


[{'query': '109580095',
  '_id': '109580095',
  '_score': 22.604383,
  'name': 'beta-globin locus control region',
  'symbol': 'HBB-LCR'}]

In [29]:
hp_to_gene = hp_gene.query('gene_id in @node_ids')
len(hp_to_gene)

142945

## Now HP to Disease Links

HP has OMIM ids for diseases... We also have some OMIM Xrefs from WikiData that we can use to merge these concepts.

In [30]:
hp_anno.query('db == "OMIM"')['db_object_id'].nunique()

7605

In [31]:
hp_anno['disease_id'] = hp_anno['db'] + ':' + hp_anno['db_object_id']

In [32]:
prev_nodes.query('label == "Disease"')['id'].nunique()

15247

In [33]:
dis_to_omim = get_curi_xrefs(prev_nodes.query('label == "Disease"'), 'OMIM')
print("Number of OMIM not-mappable to our NW: {:,}".format(len(
    set(hp_anno.query('db == "OMIM"')['disease_id']) - set(dis_to_omim['xrefs']))))

Number of OMIM not-mappable to our NW: 2,210


In [34]:
found_diseases = set(hp_anno.query('db == "OMIM"')['disease_id']) & set(dis_to_omim['xrefs'])
hp_to_dis = hp_anno.query('disease_id in @found_diseases').copy()

print("Number of OMIM mapped to our NW: {:,}".format(hp_to_dis['disease_id'].nunique()))
print("Number of annotations: {:,}".format(len(hp_to_dis)))

Number of OMIM mapped to our NW: 5,395
Number of annotations: 77,597


In [35]:
hp_to_dis['evidence_code'].value_counts()

TAS    35548
IEA    34599
PCS     7450
Name: evidence_code, dtype: int64

In [36]:
hp_to_dis = hp_to_dis.sort_values(['hp_id', 'disease_id', 'evidence_code']).drop_duplicates(subset=['disease_id', 'hp_id'])
hp_to_dis.head(4)

,db,db_object_id,db_name,qualifier,hp_id,db_reference,evidence_code,onset_modifier,frequency,sex,modifier,aspect,date_created,assigned_by,hp_name,disease_id
1570,OMIM,107480,TOWNES-BROCKS SYNDROME,NaN,HP:0000003,OMIM:107480,IEA,NaN,NaN,NaN,P,NaN,HPO:iea,NaN,Multicystic kidney dysplasia,OMIM:107480
3872,OMIM,120330,PAPILLORENAL SYNDROME,NaN,HP:0000003,OMIM:120330,TAS,NaN,NaN,NaN,P,NaN,HPO:probinson,NaN,Multicystic kidney dysplasia,OMIM:120330
7441,OMIM,143400,#143400 CONGENITAL ANOMALIES OF KIDNEY AND URI...,NaN,HP:0000003,OMIM:143400,TAS,NaN,NaN,NaN,P,NaN,HPO:skoehler,NaN,Multicystic kidney dysplasia,OMIM:143400
11308,OMIM,164210,HEMIFACIAL MICROSOMIA,NaN,HP:0000003,OMIM:164210,IEA,NaN,NaN,NaN,P,NaN,HPO:iea,NaN,Multicystic kidney dysplasia,OMIM:164210


In [37]:
hp_to_dis = hp_to_dis.merge(dis_to_omim, how='left', left_on='disease_id', right_on='xrefs').rename(columns={'id': 'nw_id'})

In [38]:
hp_to_dis.head(2)[['nw_id', 'hp_id', 'evidence_code']]

,nw_id,hp_id,evidence_code
0,DOID:0050887,HP:0000003,IEA
1,DOID:0090006,HP:0000003,TAS


## Figure out which new phenotypes need to be added

In [39]:
keep_hp_ids = set(prev_nodes.query('label == "Phenotype"')['id']) | (set(hp_to_dis['hp_id']) & set(hp_to_gene['hp_id']))

In [40]:
old_hp_ids = prev_nodes.query('label == "Phenotype"')['id']

In [41]:
keep_hp_nodes = hp_nodes.query('id in @keep_hp_ids and id not in @old_hp_ids')[['id', 'name', 'xref']]

# Standardize column name to what we have in the network
keep_hp_nodes = keep_hp_nodes.rename(columns={'xref': 'xrefs'})

# Standarize the curis to what we have in the network
keep_hp_nodes['xrefs'] = (keep_hp_nodes['xrefs'].str.replace('MSH:', 'MESH:')
                                                .str.replace('SNOMEDCT_US:', 'SNOMED:')
                                                .str.replace('MeSH:', 'MESH:'))
# Get rid of xrefs to DBs we don't already look out for
keep_hp_nodes['xrefs'] = keep_hp_nodes['xrefs'].apply(lambda s: '|'.join(sorted([x for x in str(s).split('|') 
                                                                          if x.split(':')[0] in common_curi])))
# Logic above destroys NaNs, so fix that.
keep_hp_nodes['xrefs'] = keep_hp_nodes['xrefs'].replace('', float('nan'))
keep_hp_nodes['label'] = 'Phenotype'
keep_hp_nodes.head()

,id,name,xrefs,label
5,HP:0000007,Autosomal recessive inheritance,SNOMED:258211005|UMLS:C0441748|UMLS:C4020899,Phenotype
6,HP:0000008,Abnormality of female internal genitalia,UMLS:C4025900,Phenotype
10,HP:0000012,Urinary urgency,SNOMED:75088002|UMLS:C0085606|UMLS:C3544092|UM...,Phenotype
11,HP:0000013,Hypoplasia of the uterus,SNOMED:35850006|UMLS:C0266399,Phenotype
18,HP:0000021,Megacystis,MESH:C536139|UMLS:C1855311,Phenotype


## Aggregate the new edges and save to disk

In [42]:
hp_to_dis.head(2)

,db,db_object_id,db_name,qualifier,hp_id,db_reference,evidence_code,onset_modifier,frequency,sex,modifier,aspect,date_created,assigned_by,hp_name,disease_id,nw_id,xrefs
0,OMIM,107480,TOWNES-BROCKS SYNDROME,NaN,HP:0000003,OMIM:107480,IEA,NaN,NaN,NaN,P,NaN,HPO:iea,NaN,Multicystic kidney dysplasia,OMIM:107480,DOID:0050887,OMIM:107480
1,OMIM,120330,PAPILLORENAL SYNDROME,NaN,HP:0000003,OMIM:120330,TAS,NaN,NaN,NaN,P,NaN,HPO:probinson,NaN,Multicystic kidney dysplasia,OMIM:120330,DOID:0090006,OMIM:120330


In [43]:
hp_to_dis_edges = hp_to_dis.query('hp_id in @keep_hp_ids').rename(columns={'nw_id': 'start_id', 'hp_id': 'end_id'})

hp_to_dis_edges['dsrc_type'] = hp_to_dis_edges['evidence_code'].apply(lambda e: 'computed' if e == 'IEA' else 'curated')

comp_idx = hp_to_dis_edges[hp_to_dis_edges['dsrc_type'] == 'computed'].index
hp_to_dis_edges.loc[comp_idx, 'comp_type'] = hp_to_dis_edges.loc[comp_idx, 'evidence_code']

hp_to_dis_edges['type'] = 'presents'

keep_edge_cols = ['start_id', 'end_id', 'type', 'dsrc_type', 'comp_type']

hp_to_dis_edges[keep_edge_cols].head(2)

,start_id,end_id,type,dsrc_type,comp_type
0,DOID:0050887,HP:0000003,presents,computed,IEA
1,DOID:0090006,HP:0000003,presents,curated,NaN


In [44]:
hp_to_gene_edges = hp_to_gene.query('hp_id in @keep_hp_ids').rename(columns={'hp_id': 'start_id', 'gene_id': 'end_id'})
hp_to_gene_edges['type'] = 'associated_with'
hp_to_gene_edges['dsrc_type'] = 'computed'

# from https://hpo.jax.org/app/download/annotation
# ""All phenotype terms associated with any disease that is associated with 
# variants in a gene are assigned to that gene in this file.""
# This sounds like punning to me: (A->B->C becomes A->C)
hp_to_gene_edges['comp_type'] = 'punning'

hp_to_gene_edges[keep_edge_cols].head(2)

,start_id,end_id,type,dsrc_type,comp_type
0,HP:0001250,NCBIGene:8192,associated_with,computed,punning
1,HP:0004322,NCBIGene:8192,associated_with,computed,punning


In [45]:
all_hp_edges = pd.concat([hp_to_dis_edges[keep_edge_cols], 
                       hp_to_gene_edges[keep_edge_cols]], ignore_index=True)
all_hp_edges.head(2)

,start_id,end_id,type,dsrc_type,comp_type
0,DOID:0050887,HP:0000003,presents,computed,IEA
1,DOID:0090006,HP:0000003,presents,curated,NaN


In [46]:
all_hp_edges['source'] = 'Human Phenotype Ontology'
all_hp_edges['license'] = 'custom cite no-derivatives'

In [47]:
edges_out = pd.concat([prev_edges, all_hp_edges], sort=False, ignore_index=True)

In [48]:
len(edges_out)

3793817

In [49]:
edges_out = combine_group_cols_on_char(edges_out, ['start_id', 'end_id', 'type'], sort=True, prog=False)
len(edges_out)

3785504

In [50]:
nodes_out = pd.concat([prev_nodes, keep_hp_nodes], sort=False, ignore_index=True)

In [51]:
all_edge_ids = edges_out[['start_id', 'end_id']].stack().unique()

In [52]:
nodes_filt = nodes_out.query('id in @all_edge_ids').copy()

In [53]:
edges_out.duplicated(subset=['start_id', 'end_id', 'type']).sum()

0

In [54]:
edges_out.to_csv(out_dir.joinpath('edges.csv'), index=False)
nodes_out.to_csv(out_dir.joinpath('nodes.csv'), index=False)
nodes_filt.to_csv(out_dir.joinpath('nodes_filt.csv'), index=False)

# Get a handle on these metapath semmantics

In [47]:
id_to_label = nodes_filt.set_index('id')['label'].to_dict()
#id_to_label = {**{k: 'Compound' for k in id_to_network['new_id']}, **id_to_label}


edges_out['src_label'] = edges_out['start_id'].map(id_to_label)
edges_out['tgt_label'] = edges_out['end_id'].map(id_to_label)

meta_edges = []
for row in edges_out.itertuples():
    meta_edges.append((row.src_label, row.type, row.tgt_label))
meta_edges = pd.Series(meta_edges)
meta_edges.value_counts()

(Biological Process, associated_with_BPawD, Disease)                    1798362
(Compound, associated_with_CawPW, Pathway)                               974324
(Micro RNA, regulates_NrG, Gene)                                         373942
(Disease, associated_with_DawPW, Pathway)                                286429
(Drug, associated_with_CawPW, Pathway)                                   232345
(Molecular Function, associated_with_MFawD, Disease)                     199530
(Gene, part_of_GpoPW, Pathway)                                           155800
(Gene, involved_in_GinBP, Biological Process)                            135869
(Cellular Component, associated_with_CCawD, Disease)                     126497
(Compound, increases_expression_CieG, Gene)                              123238
(Compound, decreases_expression_CdeG, Gene)                              111558
(Gene, part_of_GpoRX, Reaction)                                          101737
(Drug, increases_expression_CieG, Gene) 

In [48]:
meta_edges = meta_edges.value_counts().to_frame().reset_index()
meta_edges['start_type'] = meta_edges['index'].apply(lambda e: e[0])
meta_edges['rel'] = meta_edges['index'].apply(lambda e: e[1])
meta_edges['end_type'] = meta_edges['index'].apply(lambda e: e[2])
meta_edges = meta_edges.drop(['index'], axis=1).rename(columns={0: 'counts'})
meta_edges = meta_edges.sort_values(['start_type', 'end_type', 'counts'], ascending=[True, True, False])
meta_edges[['start_type', 'rel', 'end_type', 'counts']].to_csv(out_dir.joinpath('meta_edges.csv'), index=False)